### Task_1. GPT для цитат

In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.options.display.max_colwidth = 300

In [3]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [4]:
model_name = "bankholdup/rugpt3_song_writer"

In [5]:
df_rec = pd.read_json('dataset.jsonl', lines=True).set_index('id')
df_rec

,date,rating,text
id,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они просто очень разборчивы в друзьях ;)"
2,2004-08-30 11:25:00+00:00,25105.0,"<томатик_рад> а ты не чувствуешь красоту мира?\n<fox> честно говоря, я сейчас чувствую только отсутствие http.\n<томатик_рад> не туда смотришь, глянь вокруг!\n<fox> как я гляну, если http не работает? :/"
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаза?"" УЙДИ!!! я ХАРАКИРИ делаю!!!!!!"
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать в палату к девочкам?! - Если б мы были больные - мы б бегали к другим мальчикам"""
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с субейзом под одбц. \n<Ohtori_Akio> лучше бы мы жили в пещере с гоблинами.
...,...,...,...
463644,2020-11-26 06:12:00+00:00,698.0,"xxx: угадайте не гугля, что такое жопиздан!\nyyy: намекни хотя бы, это география или порнуха?\nxxx: это бл#%^ job is done!"
463645,2020-11-26 06:12:00+00:00,816.0,"xxx:\nПосетила шальная мысль заняться собой, жирок сбросить и подкачаться малость. Начал штудировать теорию. Выяснил, что у человека бывает три типа телосложения - эктоморфное, мезоморфное и эндоморфное. Решил с утра начать бегать. Утром проснулся, попытался встать..., понял, что у меня телослож..."
463646,2020-11-26 06:46:00+00:00,88.0,"#всебожьяроса\nxxx: Судьба айтишников вообще незавидная\nxxx: Вы должны за копейки обслуживать все компы в организации\nxxx: Вы должны за копейки поддерживать всех сотрудников организации\nxxx: Но самое главное\nxxx: Руководство получает большие бабки и строит из себя ""эффективных менеджеров""\nx..."


In [6]:
df_rec.shape

(81497, 3)

In [7]:
import re

def clear_text(text):
    clr_text = re.sub(r"<.*?>", " ", text).lower()
    clr_text = summary = re.sub(r"\s", " ", clr_text)
    return clr_text

In [8]:
df_rec["clear_text"] = df_rec["text"].apply(lambda x: clear_text(x))
df_rec

,date,rating,text,clear_text
id,,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они просто очень разборчивы в друзьях ;)","ppdv, все юниксы очень дружелюбны.. они просто очень разборчивы в друзьях ;)"
2,2004-08-30 11:25:00+00:00,25105.0,"<томатик_рад> а ты не чувствуешь красоту мира?\n<fox> честно говоря, я сейчас чувствую только отсутствие http.\n<томатик_рад> не туда смотришь, глянь вокруг!\n<fox> как я гляну, если http не работает? :/","а ты не чувствуешь красоту мира? честно говоря, я сейчас чувствую только отсутствие http. не туда смотришь, глянь вокруг! как я гляну, если http не работает? :/"
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаза?"" УЙДИ!!! я ХАРАКИРИ делаю!!!!!!","""мышка, почему у тебя такие большие глаза?"" уйди!!! я харакири делаю!!!!!!"
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать в палату к девочкам?! - Если б мы были больные - мы б бегали к другим мальчикам""","""мальчики, вы что больные, бегать в палату к девочкам?! - если б мы были больные - мы б бегали к другим мальчикам"""
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с субейзом под одбц. \n<Ohtori_Akio> лучше бы мы жили в пещере с гоблинами.,мы - как разработчики - живём с субейзом под одбц. лучше бы мы жили в пещере с гоблинами.
...,...,...,...,...
463644,2020-11-26 06:12:00+00:00,698.0,"xxx: угадайте не гугля, что такое жопиздан!\nyyy: намекни хотя бы, это география или порнуха?\nxxx: это бл#%^ job is done!","xxx: угадайте не гугля, что такое жопиздан! yyy: намекни хотя бы, это география или порнуха? xxx: это бл#%^ job is done!"
463645,2020-11-26 06:12:00+00:00,816.0,"xxx:\nПосетила шальная мысль заняться собой, жирок сбросить и подкачаться малость. Начал штудировать теорию. Выяснил, что у человека бывает три типа телосложения - эктоморфное, мезоморфное и эндоморфное. Решил с утра начать бегать. Утром проснулся, попытался встать..., понял, что у меня телослож...","xxx: посетила шальная мысль заняться собой, жирок сбросить и подкачаться малость. начал штудировать теорию. выяснил, что у человека бывает три типа телосложения - эктоморфное, мезоморфное и эндоморфное. решил с утра начать бегать. утром проснулся, попытался встать..., понял, что у меня телосложе..."
463646,2020-11-26 06:46:00+00:00,88.0,"#всебожьяроса\nxxx: Судьба айтишников вообще незавидная\nxxx: Вы должны за копейки обслуживать все компы в организации\nxxx: Вы должны за копейки поддерживать всех сотрудников организации\nxxx: Но самое главное\nxxx: Руководство получает большие бабки и строит из себя ""эффективных менеджеров""\nx...","#всебожьяроса xxx: судьба айтишников вообще незавидная xxx: вы должны за копейки обслуживать все компы в организации xxx: вы должны за копейки поддерживать всех сотрудников организации xxx: но самое главное xxx: руководство получает большие бабки и строит из себя ""эффективных менеджеров"" xxx: т...."


In [9]:
data = df_rec.loc[:, 'clear_text']

In [10]:
from sklearn.model_selection import train_test_split

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w', encoding='UTF-8')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        data += summary + "  "
    f.write(data)

In [11]:
train, test = train_test_split(data, test_size=0.15)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [12]:
print("Train dataset length: "+ str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 69272
Test dataset length: 12225


In [13]:
train

id
428854    из жж zems - а почему у вас в иерархии наверху мастер, а внизу - заказчик? наоборот же? - это чтобы было больше нормальных людей. а мудаков меньше. а то они считают, что на объекте главный - это заказчик. с тем же успехом можно иметь гнойный аппендицит, 1000 долларов и святую уверенность, что ра...
416246    mmm: сижу, проверяю сочинения на тему "моя любимая игрушка". один мальчик написал, что его любимая игрушка - майнкрафт. и что у него была проблема ее установить, потому что он не мог скачать джаву, а потом брат ему помог и теперь это его любимая игрушка.  mmm: я даже не знаю, что ему за это став...
438960                                                                                                                                                                               к новости о новых денежных банкнотах номиналом 200 и 2000 рублей:  xxx: эльдорадо: не, 200 не надо. надо 199,99. чтобы без сдачи
402247    машечка: просто в основном все эти неформальные группирро

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Downloading:   0%|          | 0.00/591 [00:00<?, ?B/s]

C:\Users\nikit\.conda\envs\NLP\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nikit\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/668 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

C:\Users\nikit\.conda\envs\NLP\lib\site-packages\transformers\data\datasets\language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [16]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_name)

Downloading:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [17]:
training_args = TrainingArguments(

    "phrase",
    evaluation_strategy = "epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    learning_rate=1e-5,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',

    )

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [19]:
trainer.train()

C:\Users\nikit\.conda\envs\NLP\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=18002, training_loss=4.106529771692289, metrics={'train_runtime': 77960.8984, 'train_samples_per_second': 0.924, 'train_steps_per_second': 0.231, 'total_flos': 4703648514048000.0, 'train_loss': 4.106529771692289, 'epoch': 2.0})

In [59]:
def generate_text(prefix):
    tokens = tokenizer(prefix, return_tensors='pt')
    size = tokens['input_ids'].shape[1]

    output = model.generate(
        **tokens,
        do_sample=False,
        max_length=size+50,
        early_stopping=True,
        length_penalty=2.0,
        repetition_penalty=8.,
        temperature=0.5,
        num_beams=3,
        no_repeat_ngram_size=5
    )

    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return prefix + result

In [60]:
print(generate_text("Саня Спи: привет=))"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Саня Спи: привет=))  xxx: а у нас на работе в отделе есть один сотрудник, который постоянно что-то делает. и вот однажды он приходит к нам с просьбой помочь ему сделать какую-нибудь работу по дому... ну мы его конечно же не стали ждать


In [61]:
print(generate_text("Махакамский спирт"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Махакамский спирт.  xxx: у нас на работе есть один чувак, который работает в одной компании с одним и тем же человеком (всё время что-то не так) yyy: а ты его знаешь? ххх: да


In [62]:
print(generate_text("Остров сокровищ"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Остров сокровищ"  xxx: у нас на работе есть один сотрудник, который работает в нашей компании. и вот однажды он приходит к нам с вопросом "а вы не могли бы мне помочь?". я ему говорю - ну как ты можешь так говорить? а


### Task_2. Обучить GPT или T5 для генерации заголовков статьей

In [28]:
from corus import load_lenta2

path = 'lenta-ru-news.csv.bz2'
records = load_lenta2(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/1914/09/16/hungarnn/',
    title='1914. Русские войска вступили в\xa0пределы Венгрии  ',
    text='Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии

In [29]:
def load_lenta_to_list(path, max_number=None):
    records = load_lenta2(path)
    texts, titles = [], []
    for i, record in enumerate(records):
        texts.append(record.text)
        titles.append(record.title)
        if not max_number is None:
            if i >= max_number-1:
                break
    return texts, titles

In [30]:
texts, titles = load_lenta_to_list(path, max_number=20000)
print(len(texts))
print(len(titles))

20000
20000


In [31]:
df = pd.DataFrame({'text':texts, 'title':titles})
df

,text,title
0,"Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты...",1914. Русские войска вступили в пределы Венгрии
1,"Министерство народного просвещения, в виду происходящих чрезвычайных событий, признало соответственным в день годовщины со дня рождения М.Ю. Лермонтова (2-го октября 1914 года) ограничиться совершением в учебных заведениях панихиды по поэту, отложив празднование юбилея до более благоприятного вр...",1914. Празднование столетия М.Ю. Лермонтова отложено
2,"Штабс-капитан П. Н. Нестеров на днях, увидев в районе Желтиева, в Галиции, летящий над нашим расположением австрийский аэроплан, собиравшийся бросить бомбы, взлетел на воздух, атаковал неприятеля и протаранил неприятельский аппарат, предотвратив жертвы в наших войсках. Сам Нестеров при этом поги...",1914. Das ist Nesteroff!
3,"Фотограф-корреспондент Daily Mirror рассказывает случай, который порадует всех друзей животных. Лейтенант бельгийской артиллерии, руководивший обороной одного из фортов Льежа, ни за что не хотел расстаться с своей собакой — бульдогом. Когда пруссаки пробрались между фортов в самый город, офицеру...",1914. Бульдог-гонец под Льежем
4,"Лица, приехавшие в Варшаву из Люблина, передают, что туда доставлен «швабский зверь» — взятый в плен прусский майор Прейскер, бывший комендант Калиша. Это — здоровый детина, с типично прусским наглым лицом и мутными глазами. Когда этого «зверя» вели с партией пленных по улице и из толпы раздалис...",1914. Под Люблином пойман швабский зверь
...,...,...
19995,"Москвич Александр Морозевич одержал черными победу над голландцем Люком ван Вели в третьем туре крупного шахматного турнира в Вейк-ан-Зее. Теперь у российского гроссмейстера 2 1/2 очка после трех туров, и он лидирует вместе со своим соотечественником Гарри Каспаровым. Экс-чемпион мира сыграл бел...",Шахматный турнире в Вейк-ан-Зее: Морозевич побеждает и догоняет Каспарова
19996,"19 ведущих высокотехнологичных компаний США объявили о создании альянса, направленного на борьбу с компьютерной преступностью. Главный метод борьбы - координация усилий по защите информации, сообщает CNN. Так, если один из членов организации обнаружит новый вирус или способ взломать защищенную к...",Компьютерные гиганты объединяются для борьбы с хакерами
19997,"В центре Москвы, вблизи станции метро ""Цветной бульвар"", по адресу Печатников переулок, 16, загорелся жилой дом старой постройки, сообщает собственный корреспондент Lenta.Ru. Возгорание возникло в подвале дома, но затем огонь быстро перекинулся выше и открытое горение началось на всех трех этажа...","Пожарные так спешили на пожар, что попали в аварию на Садовом кольце"
19998,"Две бывшие учительницы - американка Энн Бэнкрофт и норвежка Лив Арнесен - во вторник на лыжах добрались до Южного полюса, сообщает ИТАР-ТАСС. Женщины намереваются пересечь всю Антарктиду. Стартовав из района Земли Королевы Мод 64 дня назад, они преодолели 2080 километров - это примерно две тре...",Две бывшие учительницы дошли на лыжах до Южного полюса


In [32]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.1, random_state=1)

In [33]:
from datasets import Dataset, DatasetDict

ds_data = DatasetDict({
    'train': Dataset.from_pandas(df_train),
    'test': Dataset.from_pandas(df_test)
})

ds_data

DatasetDict({
    train: Dataset({
        features: ['text', 'title', '__index_level_0__'],
        num_rows: 18000
    })
    test: Dataset({
        features: ['text', 'title', '__index_level_0__'],
        num_rows: 2000
    })
})

In [34]:
max_len_text = max(map(lambda txt: len(txt.split()), ds_data['train']['text']))
max_len_tl = max(map(lambda txt: len(txt.split()), ds_data['train']['title']))
max_len_text, max_len_tl

(1111, 18)

In [35]:
max_len_text, max_len_tl = 512, 20

In [36]:
model_name_2 = "IlyaGusev/rut5_base_sum_gazeta"

In [37]:
tokenizer_2 = AutoTokenizer.from_pretrained(model_name_2)

Downloading:   0%|          | 0.00/279 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/828k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

loading file spiece.model from cache at C:\Users\nikit/.cache\huggingface\hub\models--IlyaGusev--rut5_base_sum_gazeta\snapshots\f09a08cae5d74c70e55da1a6ebb49f88c26f433b\spiece.model
loading file tokenizer.json from cache at C:\Users\nikit/.cache\huggingface\hub\models--IlyaGusev--rut5_base_sum_gazeta\snapshots\f09a08cae5d74c70e55da1a6ebb49f88c26f433b\tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at C:\Users\nikit/.cache\huggingface\hub\models--IlyaGusev--rut5_base_sum_gazeta\snapshots\f09a08cae5d74c70e55da1a6ebb49f88c26f433b\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\nikit/.cache\huggingface\hub\models--IlyaGusev--rut5_base_sum_gazeta\snapshots\f09a08cae5d74c70e55da1a6ebb49f88c26f433b\tokenizer_config.json


In [38]:
def tokenize(batch):
    tokenized_input = tokenizer_2(batch['text'], padding='max_length', truncation=True, max_length=max_len_text)
    tokenized_label = tokenizer_2(batch['title'], padding='max_length', truncation=True, max_length=max_len_tl)

    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

ds_data = ds_data.map(tokenize, batched=True, batch_size=8)

ds_data.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

  0%|          | 0/2250 [00:00<?, ?ba/s]

  0%|          | 0/250 [00:00<?, ?ba/s]

In [39]:
from transformers import DataCollatorForTokenClassification

data_collator_2 = DataCollatorForTokenClassification(tokenizer_2)

In [40]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments


model_2 = T5ForConditionalGeneration.from_pretrained(model_name_2)

Downloading:   0%|          | 0.00/766 [00:00<?, ?B/s]

loading configuration file config.json from cache at C:\Users\nikit/.cache\huggingface\hub\models--IlyaGusev--rut5_base_sum_gazeta\snapshots\f09a08cae5d74c70e55da1a6ebb49f88c26f433b\config.json
Model config T5Config {
  "_name_or_path": "cointegrated/rut5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "bos_token_id": 2,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 2,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "max_length": 200,
  "model_type": "t5",
  "num_beams": 5,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "torch_dtype": "float32",
  "

Downloading:   0%|          | 0.00/977M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at C:\Users\nikit/.cache\huggingface\hub\models--IlyaGusev--rut5_base_sum_gazeta\snapshots\f09a08cae5d74c70e55da1a6ebb49f88c26f433b\pytorch_model.bin
All model checkpoint weights were used when initializing T5ForConditionalGeneration.

All the weights of T5ForConditionalGeneration were initialized from the model checkpoint at IlyaGusev/rut5_base_sum_gazeta.
If your task is similar to the task the model of the checkpoint was trained on, you can already use T5ForConditionalGeneration for predictions without further training.


In [41]:
training_args_2 = TrainingArguments(
    "gen_title",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs= 2,
    remove_unused_columns=True, # Removes useless columns from the dataset
    save_strategy='no',
    report_to='none',
)

PyTorch: setting up devices


In [42]:
trainer_2 = Trainer(
    model=model_2,
    args=training_args_2,
    train_dataset=ds_data['train'],
    eval_dataset=ds_data['test'],
    data_collator=data_collator_2,
    tokenizer=tokenizer_2,
)

trainer_2.train()

C:\Users\nikit\.conda\envs\NLP\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
INX = 100
NMB = 5
print("TEXT: | {}".format(ds_data['test']['text'][INX]))
print("TITLE: | {}".format(ds_data['test']['title'][INX]))

In [ ]:
device = "cuda"

In [ ]:
import torch

input_text = ds_data['test']['text'][INX: INX+NMB]

with torch.no_grad():
    tokenized_text = tokenizer_2(input_text, padding='max_length', truncation=True, max_length=max_len_text, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model_2.generate(
        input_ids = source_ids,
        attention_mask = source_mask,
        max_length=512,
        num_beams=7,
        temperature = 1.3,
        repetition_penalty=1,
        length_penalty=1,
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    for i in range(NMB):
        pred = tokenizer_2.decode(generated_ids[i], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        print("Text: | {}".format(ds_data['test']['text'][INX+i][:1024]))
        print("TITLE: | {}".format(ds_data['test']['title'][INX+i]))
        print("OUTPUT: | {}".format(pred))
        print()